In [2]:
import pandas as pd
import re
import unicodedata
import os

#Dados Censo 2010
   
pasta = './por_municipios/'
arquivo = os.listdir(pasta)

fname = [f'{pasta}{arq}' for arq in arquivo[0:27] if arq.endswith(".xls")]

def file_to_df(file):
    return (
        pd.read_excel(
            file, 
            names=["municipio", "populacao2010", "geocodigo"],
            skiprows=7,
            usecols ="A:B,K", 
            skipfooter=2
        )
)
df_mun_2010 = pd.concat([file_to_df(f'{pasta}{arq}') for arq in arquivo[0:27] if arq.endswith(".xls")], ignore_index=True)

#Transformando a coluna populacao em inteiro
df_mun_2010["populacao2010"] = df_mun_2010["populacao2010"].astype(str)
df_mun_2010["populacao2010"] = df_mun_2010["populacao2010"].astype(int)

#Limpando as strings dos nomes dos municípios
df_mun_2010["municipio"] = df_mun_2010["municipio"].apply(lambda x: unicodedata.normalize("NFD", x))
df_mun_2010["municipio"] = df_mun_2010["municipio"].apply(lambda x: x.encode("ascii", "ignore"))
df_mun_2010["municipio"] = df_mun_2010["municipio"].apply(lambda x: x.decode("utf-8"))
df_mun_2010["municipio"] = df_mun_2010["municipio"].apply(lambda x: x.lower())
df_mun_2010["municipio"] = df_mun_2010["municipio"].apply(lambda x:x.strip())

#Transformando a coluna de geocódigo em string, para unificar o merge
df_mun_2010["geocodigo"] = df_mun_2010["geocodigo"].astype(str)

#Dados Censo 2022

#Ler o arquivo em excel, selecionando e renomeando as colunas a ser utilizadas
df_mun_2022 = pd.read_excel('POP2022_Municipios_20230622.xls',
                            skipfooter=35,
                            dtype=str,
                            header = [1],
                            usecols="B:E",
                            names=["uf","m_geocodigo", "municipio", "populacao2022"],
                            index_col = None
                            )

#Limpar a coluna população, retirando observações de notas de rodapé e transformando-a em número inteiro
df_mun_2022['populacao2022'] = df_mun_2022['populacao2022'].astype(str)
df_mun_2022['populacao2022'] = df_mun_2022['populacao2022'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
df_mun_2022['populacao2022'] = df_mun_2022['populacao2022'].apply(lambda x: x.replace('.','')).astype(int)

#Criar a coluna de geocódigo, com o código completo de cada município, retirando as de uf e m_geocodigo
df_mun_2022['geocodigo'] = df_mun_2022['uf'].str.cat(df_mun_2022['m_geocodigo'], sep = None)
retirar = ['uf', 'm_geocodigo']
df_mun_2022 = df_mun_2022.drop(retirar, axis=1)

#Limpando as strings dos nomes dos municípios
df_mun_2022['municipio'] = df_mun_2022['municipio'].apply(lambda x: unicodedata.normalize("NFD", x))
df_mun_2022['municipio'] = df_mun_2022['municipio'].apply(lambda x: x.encode("ascii", "ignore"))
df_mun_2022['municipio'] = df_mun_2022['municipio'].apply(lambda x: x.decode("utf-8"))
df_mun_2022['municipio'] = df_mun_2022['municipio'].apply(lambda x: x.lower())
df_mun_2022['municipio'] = df_mun_2022['municipio'].apply(lambda x:x.strip())

#Juntando os dois dataframes
df_merge = pd.merge(df_mun_2010,
                    df_mun_2022,
                    how = 'outer',
                    left_on = ['geocodigo'],
                    right_on=['geocodigo'],
                    suffixes = ('2010','2022')
                    )
df_merge['populacao2022'] = df_merge['populacao2022'].astype(float)
df_merge = df_merge.drop_duplicates()
df_merge.head()

,municipio2010,populacao2010,geocodigo,municipio2022,populacao2022
0,alta floresta d'oeste,24285.0,1100015,alta floresta d'oeste,21558.0
1,alto alegre dos parecis,12808.0,1100379,alto alegre dos parecis,13023.0
2,alto paraiso,17132.0,1100403,alto paraiso,17140.0
3,alvorada d'oeste,16808.0,1100346,alvorada d'oeste,13092.0
4,ariquemes,89934.0,1100023,ariquemes,100896.0
